<center>
    <h1> INF285 - Computación Científica </h1>
    <h2> Tarea 2 - Código Base</h2>
    <h2> [S]cientific [C]omputing [T]eam </a> </h2>
    <h2> Version: 1.00</h2>
</center>

# No debe utilizar bibliotecas adicionales.

#**IMPORTANTE:** Se comentaron todos los prints que existían (incluso en las funciones del profesor).

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import random
import numpy as np
from scipy import linalg
from ipywidgets import interact

In [ ]:
# Firma función 1
def load_data(emotion):
    """
    Parameters
    ----------
    emotion       : (string) String con el nombre de la emoción a trabajar. 
    Returns
    -------
    IMAGES        : (ndarray 3d) Numpy array con todas las imágenes 
                                 pertenecientes a la emoción ingresada. 
                                 Shape:(cant_imagenes, pixeles_y, pixeles_x).
    """
    img_folder = './facial_expressions/'+emotion
    img_dir = os.listdir(img_folder)
    cant_img = len(img_dir)
    pixels_number = 48 # (48*48)
    IMAGES = np.zeros((cant_img, pixels_number, pixels_number))
    row = 0
    for image in img_dir:
        image_path = os.path.join(img_folder, image)
        originalImage = cv2.imread(image_path)
        grayImage = cv2.cvtColor(originalImage, cv2.COLOR_BGR2GRAY)
        IMAGES[row, :, :] = np.array(grayImage)
        row += 1
    return IMAGES
"""
Ejemplo de ejecución función anterior para cargar imágenes felices.
emotion = 'happy'
IMAGES = load_data(emotion)
"""
#firma función 2
def total_error(IMAGES, V_l, Y_l, mu):
    """
    Parameters
    ----------
    IMAGES        : (ndarray 3d) Numpy array con todas las imágenes 
                                 pertenecientes a la emoción ingresada. 
                                 Shape:(cant_imagenes, pixeles_y, pixeles_x).
    V_l            : (ndarray 2d)-Numpy array con 
                     los l componentes principales.
                     hint>Shape:(pixeles_y * pixeles_x, l).
    Y_l            : (ndarray 2d)-Numpy array con los coeficientes
                     de los l componentes principales.
                     hint>Shape:(cant_imagenes, l).
    mu             : (ndarray 1d)-Numpy array con el
                     promedio por columna de la matriz X.
                     hint>Shape:(pixeles_y * pixeles_x,).
    Returns
    -------
    error_total     : (float) error total promedio de todas las imágenes.
    """
    m, n1, n2 = IMAGES.shape
    error = np.zeros(m)
    for k in range(m):
        x_k = reconstruct_x_k(V_l, Y_l[k,:], mu)
        #print(Y_l[k,:].shape)
        Im_k = build_Image_from_x_k(x_k, n1, n2)
        error[k] = np.linalg.norm(IMAGES[k,:,:]-Im_k)
    error_total = np.mean(error)
    return error_total
#firma función 3
def compute_all_errors_and_compression_rates(IMAGES):
    """
    Parameters
    ----------
    IMAGES        : (ndarray 3d) Numpy array con todas las imágenes 
                                 pertenecientes a la emoción ingresada. 
                                 Shape:(cant_imagenes, pixeles_y, pixeles_x).
    
    Returns
    -------
    errors_and_compressions   : (ndarray 2d) Numpy array donde la primera fila corresponde
                                a los errores de reconstrucción asociados a todas las imágenes
                                La segunda fila corresponde los compression_ratio asociados
                                a todas las imágenes.
                                
                                Cada columna corresponde al error o compresión usando un 
                                número arbitrario de componentes principales.
                                de componentes principales.
    n_range                   : (ndarray 1d) Numpy array donde cada columna es la cantidad
                                de componentes principales utilizadas.
    """
    m, n1, n2 = IMAGES.shape
    X = build_X(IMAGES)
    n_range1 = np.array([1,2,4,8,16,32,64])
    n_range2 = np.logspace(7, np.log2(2100), 10, base=2, dtype=int, endpoint=True)
    n_range = np.concatenate((n_range1, n_range2, np.array([2304])))
    n_samples = len(n_range)
    errors_and_compressions = np.zeros((2, n_samples))
    V_n, Y_n, mu = DATA_to_PCA(X, n1*n2)
    for k, l in zip(np.arange(n_samples),n_range):
        #print('Processing l=',l)
        V_l = V_n[:,:l]  
        Y_l = Y_n[:,:l]
        error_l = total_error(IMAGES, V_l, Y_l, mu)
        camp_l = calc_compression_ratio(m, n1, n2, l)
        errors_and_compressions[0, k] = error_l
        errors_and_compressions[1, k] = camp_l
        #print('Processed l=',l)
    return errors_and_compressions, n_range

In [ ]:
#Firma función 4
def build_X(IMAGES):
    """
    Parameters
    ----------
                                 pertenecientes a la emoción ingresada. 
                                 hint>Shape:(cant_imagenes, pixeles_y, pixeles_x).
    Returns
    -------
    X             : (ndarray 2d) Numpy array con todas las imágenes de IMAGES
                                 como vectores fila de una matriz.
                                 hint>Shape:(cant_imagenes, pixeles_y * pixeles_x).
    """
    # Acá va su firma
    #Obtenemos los datos del array IMAGES
    cantImagenes, pixelX, pixelY = np.shape(IMAGES)
    X = np.empty([cantImagenes, pixelX*pixelY])

    #Se crea el array con los datos correspondientes, usando flatten
    for image in range(0, cantImagenes):
      X[image] = IMAGES[image].flatten()

    return X


In [ ]:
#Firma función 3
def DATA_to_PCA(X, l):
    """
    Parameters
    ----------
    X             : (ndarray 2d) Numpy array con todas las imágenes de IMAGES
                                 como vectores fila de una matriz.
                                 hint>Shape:(cant_imagenes, pixeles_y * pixeles_x).
    l              : (int)       Número de componentes principales.
    Returns
    -------
    V_l            : (ndarray 2d)-Numpy array con 
                     los l componentes principales.
                     hint>Shape:(pixeles_y * pixeles_x, l).
    Y_l            : (ndarray 2d)-Numpy array con los coeficientes
                     de los l componentes principales.
                     hint>Shape:(cant_imagenes, l).
    mu             : (ndarray 1d)-Numpy array con el
                     promedio por columna de la matriz X.
                     hint>Shape:(pixeles_y * pixeles_x,).
    """
    # Acá va su firma
    #Se obtiene el valor de μ
    mu = np.mean(X, axis=0)
    matrizZ = X-mu

    #Ejecutamos la SVD reducida (ya que es lo recomendable según lo que aparece en el anexo)
    matrizU, matrizS, matrizV = linalg.svd(matrizZ, full_matrices = False)

    # Se obtienen los valores transpuestos de V_l con los l componentes y también Y_l
    V_l = matrizV.T
    V_l = V_l[:,:l]  
    Y_l =  np.dot(matrizZ, V_l)
    Y_l = Y_l[:,:l]

    
    return V_l, Y_l, mu

In [ ]:
#Firma función 5
def PCA_to_DATA(V_l, Y_l, mu):
    """
    Parameters
    ----------
    V_l            : (ndarray 2d)-Numpy array con 
                     los l componentes principales.
                     hint>Shape:(pixeles_y * pixeles_x, l).
    Y_l            : (ndarray 2d)-Numpy array con los coeficientes
                     de los l componentes principales.
                     hint>Shape:(cant_imagenes, l).
    mu             : (ndarray 1d)-Numpy array con el
                     promedio por columna de la matriz X.
                     hint>Shape:(pixeles_y * pixeles_x,). 
    Returns
    -------
    X_l            : (ndarray 2d)-Numpy array con la matriz X_l
                     correspondiente a la reconstrucción de X usando
                     l componentes principales.
                     hint>Shape:(cant_imagenes, pixeles_y * pixeles_x)
    """
    # Acá va su firma

    #Se obitene la traspuesta de V_l y aplicamos el producto punto
    V_l = V_l.T
    matrizZ = np.dot(Y_l, V_l)
    X_l = matrizZ + mu

    return X_l

In [ ]:
# Firma función 6
def reconstruct_x_k(V_l, y_k, mu):
    """
    Parameters
    ----------
    V_l            : (ndarray 2d)-Numpy array con 
                     los l componentes principales.
                     hint>Shape:(pixeles_y * pixeles_x, l).
    y_k            : (ndarray 1d)-Numpy array con la fila
                     k de la matriz Y_l.
                     hint>Shape:(l,)
                     
    Returns
    -------
    x_k            : (ndarray 1d)-Numpy array con
                     la reconstrucción de la imagen en la 
                     fila k de la matriz X usando l componentes principales.
                     hint>Shape:(pixeles_y * pixeles_x, )
    """
    # Acá va su firma

    #Se reconstruye la imagen (tomando en cuenta la función 6)
    V_l = V_l.T
    Z = np.dot(y_k, V_l)
    x_k = Z + mu

    return x_k

In [ ]:
# Firma función 7
def build_Image_from_x_k(x_k, n1, n2):
    """
    Parameters
    ----------
    x_k            : (ndarray 1d)-Numpy array con
                     la reconstrucción de la imagen en la 
                     fila k de la matriz X usando l componentes principales.
    n1             : (int) cantidad de pixeles x por imagen.
    n2             : (int) cantidad de pixeles y por imagen.
                     
    Returns
    -------
    Im            :  (ndarray 2d)-Numpy array con
                     la la imagen contenida en x_k, luego de aplicar un reshape.
                     Hint>Shape:(pixeles_y, pixeles_x)
    """
    #Acá va su firma

    #Se utiliza el reshape de numpy para actualizar la imagen
    Im = np.reshape(x_k, (n1,n2))

    return Im



In [ ]:
# Firma función 8
def calc_compression_ratio(m, n1, n2, l):
    """
    Parameters
    ----------
    m          : (int) Cantidad total de imágenes.
    n1         : (int) Cantidad de pixeles x por imagen.
    n2         : (int) Cantidad de pixeles y por imagen.
    l          : (int) Cantidad de componentes principales.
    Returns
    -------
    compression_ratio  : (Float)
                         Compression ratio.
    """
    #Acá va su firma

    #Se calcula la memoria original y la memoria comprimida
    #Si bien existen varias formas de realizarlo, como el del siguiente link: https://towardsdatascience.com/face-dataset-compression-using-pca-cddf13c63583
    #En la sección: "Compression Ratio", el cálculo se hará explícitamente como está calculado en los apuntes (ya que así nos piden calcularlo)
    memoriaOriginal = m*n1*n2
    memoriaComprimida = m*l + n1*n2*l + n1*n2

    #Se calcula el ratio según la fórmula entregada
    compression_ratio = (1-(memoriaComprimida/memoriaOriginal))*100



    return compression_ratio

In [ ]:
# Ejecutar una vez implementadas todas las funciones.
# Hint: El correcto funcionamiento de esta función indica que probablemente sus funciones esten bien implementadas. 
# Firma función 9
def plot_comparison(IMAGES, errors_and_compressions, n_range, j, k, V_n, Y_n, mu):
    all_errors = errors_and_compressions[0,:]
    all_compressions_rates = errors_and_compressions[1,:]
    
    m, n1, n2 = IMAGES.shape
    plt.figure(figsize=(10,10))
    ax = plt.subplot(2,2,1)
    plt.plot(n_range, all_compressions_rates,'.', label='PCA')
    plt.plot(n_range, np.zeros_like(n_range),'-', label='Sin PCA')
    plt.plot(n_range[j], all_compressions_rates[j],'r.',ms=10)
    plt.title('Análisis de Tasa de Compresión')
    plt.xlabel('Número de componentes principales')
    plt.grid(True)
    plt.legend(loc='best')

    ax = plt.subplot(2,2,2)
    plt.plot(n_range, all_errors,'.')
    plt.plot(n_range[j], all_errors[j],'r.',ms=10)
    plt.title('Error medio')
    plt.grid(True)
    plt.xlabel('Número de componentes principales')

    ax = plt.subplot(2,2,3)
    plt.imshow(IMAGES[k], cmap="gray")
    plt.title('Imagen original')

    ax = plt.subplot(2,2,4)
    x_k = reconstruct_x_k(V_n[:,:n_range[j]], Y_n[k,:n_range[j]], mu)
    #print(Y_n[k,:n_range[j]].shape)
    #print(x_k.shape)
    Im_l = build_Image_from_x_k(x_k, n1, n2)
    plt.imshow(Im_l, cmap="gray")
    plt.title('Imagen reconstruida con '+str(n_range[j])+' componentes principales')
    plt.show()

#DESCOMENTAR LAS SIGUIENTES LINEAS SI SE REQUIERE EJECUTAR EL CÓDIGO BASE

'''
IMAGES = load_data("happy")
errors_and_compressions, n_range = compute_all_errors_and_compression_rates(IMAGES)
m, n1, n2 = IMAGES.shape
X = build_X(IMAGES)
V_n, Y_n, mu = DATA_to_PCA(X, n1*n2)

data_visualization = lambda j, k: plot_comparison(IMAGES, errors_and_compressions, n_range, j, k, V_n, Y_n, mu)
interact(data_visualization, j = (0,len(n_range)-1,1), k = (0,m-1,1))
'''